In [12]:
#Import relevant libraries
import io
import itertools
import sklearn.metrics
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard, LambdaCallback
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
# import datetime
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from tensorboard.plugins.hparams import api as hp

In [13]:
#Dataset encoding explaination.
#image labelling - 0=glasses/sunglasses. 1= trousers/jeans. 3= shoes

In [14]:
#Import the datasets and preprocess

data_train = np.load(r'Full Dataset/Trousers & Jeans - All - Train.npz')
data_validation = np.load(r'Full Dataset/Trousers & Jeans - All - Validation.npz')
data_test = np.load(r'Full Dataset/Trousers & Jeans - All - Test.npz')


#t extract the arrays from dataset into input(images) and target(labels)
images_train = data_train['images']
labels_train = data_train['labels']

images_val = data_validation['images']
labels_val = data_validation['labels']


images_test = data_test['images']
labels_test = data_test['labels']

#Pixel-wise normalization of the training, validation and testing data
images_train = images_train/255.0
images_val  = images_val/255.0
images_test = images_test/255.0







In [15]:
#Define the hyperparameters

BATCH_SIZE = 64
EPOCHS = 15

#Define the hyperparamets to tune and the variations we want to test.
HP_FILTER_SIZE = hp.HParam('filter_size', hp.Discrete([3,5,7]))
HP_FILTER_NUM = hp.HParam('filter_num', hp.Discrete([32, 64, 96, 128]))


METRIC_ACCURACY = 'accuracy'

#Log the hyperparameter with the file writer
with tf.summary.create_file_writer('Logs/ModelG 1/hparam_tuning/').as_default():
    hp.hparams_config(
        hparams=[HP_FILTER_SIZE, HP_FILTER_NUM], 
        metrics= [hp.Metric(METRIC_ACCURACY, display_name='Accuracy')]
    )



In [16]:
def train_model(hparams,session_num):
    
    
#Create and train the model


    
    model = Sequential([
                Conv2D(32, 3, activation= 'relu', input_shape=(120,90,3)),
                MaxPooling2D(pool_size=(2,2)),
                Conv2D(64, 3, activation= 'relu'),
                MaxPooling2D(pool_size=(2,2)),
                Flatten(),
                Dense(16, activation = 'relu'),
                Dropout(0.2),
                Dense(4)
    ])


#describe the loss function
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

#Compile the model
    model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])
    log_dir = f"Logs/ModelG 1/fit/run-{session_num}"

    
        
        
        
#Plot_to_image

    def plot_confusion_matrix(cm, class_names):
 
        figure = plt.figure(figsize=(8, 8))
        plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
        plt.title("Confusion matrix")
        plt.colorbar()
        tick_marks = np.arange(len(class_names))
        plt.xticks(tick_marks, class_names, rotation=45)
        plt.yticks(tick_marks, class_names)

        # Compute the labels from the normalized confusion matrix.
        labels = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)

        # Use white text if squares are dark; otherwise black.
        threshold = cm.max() / 2.
        for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
            color = "white" if cm[i, j] > threshold else "black"
            plt.text(j, i, labels[i, j], horizontalalignment="center", color=color)

        plt.tight_layout()
        plt.ylabel('True label')
        plt.xlabel('Predicted label')
        return figure
    



    def plot_to_image(figure):
       
        # Save the plot to a PNG in memory.
        buf = io.BytesIO()
        plt.savefig(buf, format='png')
        # Closing the figure prevents it from being displayed directly inside
        # the notebook.
        plt.close(figure)
        buf.seek(0)
        # Convert PNG buffer to TF image
        image = tf.image.decode_png(buf.getvalue(), channels=4)
        # Add the batch dimension
        image = tf.expand_dims(image, 0)
        return image


        
    # log the confusion matrix as a heatmap
    file_writer_cm = tf.summary.create_file_writer(log_dir + '/cm')


    def log_confusion_matrix(epoch, logs):
        # Use the model to predict the values from the validation dataset.
        val_pred_raw = model.predict(images_val)
        val_pred = np.argmax(val_pred_raw, axis=1)

        # Calculate the confusion matrix.
        cm = confusion_matrix(labels_val, val_pred)
        # Log the confusion matrix as an image summary.
        figure = plot_confusion_matrix(cm, class_names=["Trouser Male", "Trouse Female", "Jeans Male", "Jeans Female"])
        cm_image = plot_to_image(figure)

        # Log the confusion matrix as an image summary.
        with file_writer_cm.as_default():
            tf.summary.image("Confusion_matrix", cm_image, step=epoch)

    # Define the per-epoch callback.
    cm_callback = LambdaCallback(on_epoch_end=log_confusion_matrix)


    tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)


    early_stopping = EarlyStopping(
        monitor= 'val_loss',
        mode = 'auto',
        min_delta = 0,
        patience = 2,
        verbose = 0,
        restore_best_weights = True
    )


#Train the model
    model.fit(
        images_train,
        labels_train,
        epochs = EPOCHS,
        batch_size = BATCH_SIZE,
        callbacks = [tensorboard_callback, cm_callback, early_stopping],
        validation_data = (images_val, labels_val),
        verbose =2
    )

    _,accuracy = model.evaluate(images_val, labels_val)

    model.save(f"saved_model/ModelG 1/fit/run-{session_num}")


    return accuracy



In [17]:
#Log the hyperparameter to the TensorBoard



def run(log_dir, hparams,session_num):
    hyperparameter_writer = tf.summary.create_file_writer(log_dir)
    
    
    with hyperparameter_writer.as_default():
        hp.hparams(hparams) #record the values used in this trial
        accuracy = train_model(hparams, session_num)
        tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)
        
        
        
        
        
#Train the model with the different hyperparameters
session_num = 1
for filter_size in HP_FILTER_SIZE.domain.values:
    for filter_num in HP_FILTER_NUM.domain.values:
        hparams ={
        HP_FILTER_SIZE: filter_size,
        HP_FILTER_NUM : filter_num
        }
        
        run_name = "run-%d" % session_num
        print('---Starting trial: %s' % run_name)
        print({h.name: hparams[h] for h in hparams})
        run('Logs/ModelG 1/hparam_tuning/' + run_name, hparams, session_num)
        
        session_num += 1
       
        

       
        
        
        








---Starting trial: run-1
{'filter_size': 3, 'filter_num': 32}
Epoch 1/15
16/16 [==============================] - 1s 63ms/step
63/63 - 31s - loss: 1.3641 - accuracy: 0.3244 - val_loss: 1.2285 - val_accuracy: 0.4160 - 31s/epoch - 492ms/step
Epoch 2/15
16/16 [==============================] - 1s 60ms/step
63/63 - 27s - loss: 1.1850 - accuracy: 0.4301 - val_loss: 1.1461 - val_accuracy: 0.5220 - 27s/epoch - 430ms/step
Epoch 3/15
16/16 [==============================] - 1s 62ms/step
63/63 - 27s - loss: 1.1394 - accuracy: 0.4865 - val_loss: 1.0906 - val_accuracy: 0.5460 - 27s/epoch - 425ms/step
Epoch 4/15
16/16 [==============================] - 1s 59ms/step
63/63 - 26s - loss: 1.0979 - accuracy: 0.5252 - val_loss: 1.0901 - val_accuracy: 0.5540 - 26s/epoch - 420ms/step
Epoch 5/15
16/16 [==============================] - 1s 61ms/step
63/63 - 27s - loss: 1.0830 - accuracy: 0.5375 - val_loss: 1.0665 - val_accuracy: 0.5800 - 27s/epoch - 425ms/step
Epoch 6/15
16/16 [==============================

INFO:tensorflow:Assets written to: saved_model/ModelG 1/fit/run-1\assets


---Starting trial: run-2
{'filter_size': 3, 'filter_num': 64}
Epoch 1/15
16/16 [==============================] - 1s 36ms/step
63/63 - 17s - loss: 1.3993 - accuracy: 0.2515 - val_loss: 1.3868 - val_accuracy: 0.2200 - 17s/epoch - 275ms/step
Epoch 2/15
16/16 [==============================] - 1s 38ms/step
63/63 - 16s - loss: 1.3862 - accuracy: 0.2565 - val_loss: 1.3869 - val_accuracy: 0.2200 - 16s/epoch - 261ms/step
Epoch 3/15
16/16 [==============================] - 1s 38ms/step
63/63 - 17s - loss: 1.3862 - accuracy: 0.2565 - val_loss: 1.3873 - val_accuracy: 0.2200 - 17s/epoch - 271ms/step
16/16 [==============================] - 1s 42ms/step - loss: 1.3868 - accuracy: 0.2200
INFO:tensorflow:Assets written to: saved_model/ModelG 1/fit/run-2\assets


INFO:tensorflow:Assets written to: saved_model/ModelG 1/fit/run-2\assets


---Starting trial: run-3
{'filter_size': 3, 'filter_num': 96}
Epoch 1/15
16/16 [==============================] - 1s 41ms/step
63/63 - 20s - loss: 1.3803 - accuracy: 0.3449 - val_loss: 1.2922 - val_accuracy: 0.4880 - 20s/epoch - 321ms/step
Epoch 2/15
16/16 [==============================] - 1s 40ms/step
63/63 - 18s - loss: 1.2493 - accuracy: 0.4473 - val_loss: 1.1802 - val_accuracy: 0.5560 - 18s/epoch - 288ms/step
Epoch 3/15
16/16 [==============================] - 1s 52ms/step
63/63 - 20s - loss: 1.1042 - accuracy: 0.5165 - val_loss: 1.0778 - val_accuracy: 0.5440 - 20s/epoch - 322ms/step
Epoch 4/15
16/16 [==============================] - 1s 47ms/step
63/63 - 23s - loss: 1.0576 - accuracy: 0.5347 - val_loss: 1.0296 - val_accuracy: 0.5980 - 23s/epoch - 366ms/step
Epoch 5/15
16/16 [==============================] - 1s 46ms/step
63/63 - 22s - loss: 1.0242 - accuracy: 0.5402 - val_loss: 1.0230 - val_accuracy: 0.5680 - 22s/epoch - 353ms/step
Epoch 6/15
16/16 [==============================

...

INFO:tensorflow:Assets written to: saved_model/ModelG 1/fit/run-10\assets


---Starting trial: run-11
{'filter_size': 7, 'filter_num': 96}
Epoch 1/15
16/16 [==============================] - 1s 38ms/step
63/63 - 21s - loss: 1.1298 - accuracy: 0.5077 - val_loss: 0.9861 - val_accuracy: 0.5860 - 21s/epoch - 330ms/step
Epoch 2/15
16/16 [==============================] - 1s 39ms/step
63/63 - 18s - loss: 0.9296 - accuracy: 0.6001 - val_loss: 0.9820 - val_accuracy: 0.5680 - 18s/epoch - 288ms/step
Epoch 3/15
16/16 [==============================] - 1s 39ms/step
63/63 - 17s - loss: 0.8483 - accuracy: 0.6414 - val_loss: 0.8566 - val_accuracy: 0.6240 - 17s/epoch - 277ms/step
Epoch 4/15
16/16 [==============================] - 1s 38ms/step
63/63 - 18s - loss: 0.7745 - accuracy: 0.6811 - val_loss: 0.7943 - val_accuracy: 0.6660 - 18s/epoch - 280ms/step
Epoch 5/15
16/16 [==============================] - 1s 38ms/step
63/63 - 18s - loss: 0.7329 - accuracy: 0.6923 - val_loss: 0.8627 - val_accuracy: 0.6280 - 18s/epoch - 287ms/step
Epoch 6/15
16/16 [=============================

INFO:tensorflow:Assets written to: saved_model/ModelG 1/fit/run-11\assets


---Starting trial: run-12
{'filter_size': 7, 'filter_num': 128}
Epoch 1/15
16/16 [==============================] - 1s 43ms/step
63/63 - 21s - loss: 1.2866 - accuracy: 0.3754 - val_loss: 1.1015 - val_accuracy: 0.5560 - 21s/epoch - 337ms/step
Epoch 2/15
16/16 [==============================] - 1s 45ms/step
63/63 - 23s - loss: 1.0562 - accuracy: 0.5417 - val_loss: 0.9894 - val_accuracy: 0.6360 - 23s/epoch - 373ms/step
Epoch 3/15
16/16 [==============================] - 1s 43ms/step
63/63 - 20s - loss: 0.9734 - accuracy: 0.5677 - val_loss: 0.9179 - val_accuracy: 0.6280 - 20s/epoch - 315ms/step
Epoch 4/15
16/16 [==============================] - 1s 40ms/step
63/63 - 19s - loss: 0.9035 - accuracy: 0.6166 - val_loss: 0.8496 - val_accuracy: 0.6740 - 19s/epoch - 300ms/step
Epoch 5/15
16/16 [==============================] - 1s 39ms/step
63/63 - 18s - loss: 0.8565 - accuracy: 0.6394 - val_loss: 0.8526 - val_accuracy: 0.6640 - 18s/epoch - 285ms/step
Epoch 6/15
16/16 [============================

INFO:tensorflow:Assets written to: saved_model/ModelG 1/fit/run-12\assets


In [21]:
%load_ext tensorboard
%tensorboard --logdir 'Logs/ModelG 1/hparam_tuning'


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [22]:
%load_ext tensorboard
%tensorboard --logdir 'Logs/ModelG 1/fit'

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
